#### **RNN for Named Entity Recognition**

The NLP task [Named Entity Recognition](https://en.wikipedia.org/wiki/Named-entity_recognition) (NER) is to classify named entity's within a corpus into predefined categories.

We explore the use of a recurrent architecture to perform NER. The dataset used is the MIT-Restaurants dataset. The tagging of the dataset is in the [IOB2](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)) format.

Note: Framework based off https://github.com/lingo-mit/6864-hw2/blob/master/6864_hw2.ipynb

In [0]:
import numpy as np

import time
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

import util


device = "cuda" if torch.cuda.is_available() else "cpu"
assert device == "cuda" # use a gpu!

#### **Setup**

Read in the dataset

In [0]:
train_data = util.read_file_txt("../data/mit_restaurants-train.dat", type = "word")
train_tags = util.read_file_txt("../data/mit_restaurants-train.tag", type = "word")

test_data = util.read_file_txt("../data/mit_restaurants-test.dat", type = "word")
test_tags = util.read_file_txt("../data/mit_restaurants-test.tag", type = "word")

print('number of training samples:', len(train_data))
print('number of testing samples:',  len(test_data))
# print('average sentence length in training data', (np.mean([len(sent) for sent in train_data])))
print()

print('the first few sentences are:', train_data[0:3])
print('and their cp\'ding named entity sequences are: ', str(train_tags[0:3]))
print()

#### **Data Preprocessing**

In [0]:
# helper functions and more data preprocessing before we move on to implementing our models.

# from train data, collect all unique word types as a set and add 'UNK' to it (unseen words in test data will be turned into 'UNK')
vocab_set = list(set([word for sent in train_data for word in sent])) + ['UNK']
num_vocabs = len(vocab_set)
print("number of word types (including 'UNK'):", num_vocabs)
print("the first couple and last couple of words in the vocabulary set:", vocab_set[0:2] +  vocab_set[-2:])

vocab2id = {v : i for i, v in enumerate(vocab_set)}

#  collect all tag (class) types and assign an unique id to each of them. (here there won't be a unseen tag type in test data)
tag_set = list(set([tag for tag_seq in train_tags for tag in tag_seq]))
num_tags = len(tag_set)
print("number of tag types:", num_tags)
print()

# assign each tag type a unique id, also create the inverse dict of tag2id (required during evaluation)
tag2id = {t : i for i, t in enumerate(tag_set)} 
id2tag = {i : t for t, i in tag2id.items()}

# apply one-hot encoding to data.
train_data_oh_list = [util.one_hot_encoding(sent, vocab2id, vocab_set) for sent in train_data]
# print("oh data[0] - len:", len(train_data_oh_list[0]), "shape:", train_data_oh_list[0].shape)

# transform tag names into ids
train_tags_id_list = [util.encoding_idx(tag_seq, tag2id) for tag_seq in train_tags]
# print("list len, tag data:", len(train_tags_id_list))

# train_data_oh_list should now be a list of 2d-tensors, each has shape (sent_len, num_vocabs)
# Note that to utilize the `shape` attribute, each element in the list should already be a torch tensor.
print("first sentence has shape: %s" % str(train_data_oh_list[0].shape))
print("fifth sentence has shape: %s" % str(train_data_oh_list[4].shape))

# train_tags_id_list is a list of 1d-tensors, each that has shape (sent_len,)
print("first tag sequence has shape: %s" % train_tags_id_list[0].shape)
print("fifth tag sequence has shape: %s" % train_tags_id_list[4].shape)
print()


# Apply same conversion to test dataset.
test_data_oh_list = [util.one_hot_encoding(sent, vocab2id, vocab_set) for sent in test_data]
test_tags_id_list = [util.encoding_idx(tag_seq, tag2id) for tag_seq in test_tags]
# print("list len, oh test:", len(test_data_oh_list))
# print("list len, tag test:", len(test_tags_id_list))
# print("first sentence has shape: %s" % str(test_data_oh_list[0].shape))
# print("fifth sentence has shape: %s" % str(test_data_oh_list[4].shape))


#### **RNN**

We implement a vanilla RNN from scratch, then train it and evaluate its performance on the NER task.

The RNN is implemented using [Elman](https://en.wikipedia.org/wiki/Recurrent_neural_network#Elman_networks_and_Jordan_networks) units and the $tanh$ non-linear activation function. The input size was set to the vocbulary size of the dataset, the size of the hidden state was set to $128$ and the output size was set to the number of tags in the dataset.

##### **RNN architecture**

In [0]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(RNN, self).__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size

    self.hidden = nn.Linear(self.input_size + self.hidden_size, self.hidden_size)
    self.output = nn.Linear(self.hidden_size, self.output_size)
        

  def forward(self, x, hidden):
    """
    The `forward` function should just perform one step of update and output logits before softmax.
    `x` is a 2d-tensor of shape (1, input_size); `hidden` is another 2d-tensor of shape (1, hidden_size), representing the hidden state of
    the previous time step.
    """

    combined = torch.cat( (x, hidden), dim = 1)
    hidden = torch.tanh(self.hidden(combined))
    output = self.output(hidden)

    return output, hidden
    

  def init_state(self):
    """ Use to initialize hidden state everytime before running a sentence. """
    return torch.zeros(1, self.hidden_size).to(device)

##### **Training**

The function $train\_one\_sample$, takes a (sentence tensor, tag tensor) pair as input and does one step of the gradient update.

In [0]:
learning_rate = 1e-3
rnn_hidden_size = 128


model = RNN(input_size = num_vocabs, hidden_size = rnn_hidden_size, output_size = num_tags).to(device)
 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)


"""
Run through a sentence, generate output, compute loss, and perform one gradient update
Sentence and tag are represented as a 2d-tensor `sent_tensor` and a 1d-tensor `tag_tensor`, respectively.
"""
def train_one_sample(model, sent_tensor, tag_tensor):
  hidden = model.init_state() # initialize hidden state
  loss = 0.00

  outputs = torch.zeros( (1, num_tags)).to(device)

  for idx in range(sent_tensor.shape[0]):
    outputs, hidden = model(sent_tensor[idx].reshape(1, sent_tensor.shape[1]), hidden)
    loss = loss + criterion(outputs, torch.LongTensor([tag_tensor[idx]]).to(device))
      
  loss = loss / len(tag_tensor)   # average the loss over all tags in the sentance

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  return outputs, loss.item()


# main training loop for the rnn
num_epochs = 10
iter_count = 0
print_every = 2500
plot_every = 50

# Keep track of losses for plotting
current_loss = 0
all_losses = []
b_calc_grad_norm = True # set to true if we want to analyze how the gradient changes over training

if b_calc_grad_norm:
  current_grad_norm = 0
  all_grad_norms = []


start = time.time()

model.train()
 
for idx_epoch in range(num_epochs):
  for sent_tensor, tag_tensor in zip(train_data_oh_list, train_tags_id_list):
    sent_tensor = sent_tensor.to(device)
    tag_tensor = tag_tensor.to(device)

    output, loss = train_one_sample(model, sent_tensor, tag_tensor)
    current_loss += loss
    
    if b_calc_grad_norm:
      grad = torch.FloatTensor( (1, 1)).to(device)
      
      for param in model.parameters():
        grad = torch.cat( (grad, torch.reshape(param.grad, (-1, ))))
      
      current_grad_norm += torch.norm(grad.to("cpu"), 2)


    if iter_count % print_every == 0:      
      if b_calc_grad_norm:
        print("epoch: {}, iteration: {}, time: {}, loss: {:0.4f}, grad_norm: {:0.4f}".format(idx_epoch, iter_count, util.time_since(start), loss, torch.norm(grad.to("cpu"), 2)))
      else:
        print("epoch: {}, iteration: {}, time: {}, loss: {:0.4f}".format(idx_epoch, iter_count, util.time_since(start), loss))


    # add current loss avg to list of losses
    if iter_count % plot_every == 0 and iter_count > 0:
      all_losses.append(current_loss / plot_every)
      current_loss = 0
      
      if b_calc_grad_norm: 
        all_grad_norms.append(current_grad_norm / plot_every)
        current_grad_norm = 0


    iter_count += 1

##### **Evaluation**

We plot the learning curve. We also evaluate the model on the test data.

###### **Learning Curve**

We plot the learning curve.

In [0]:
# plot the learning curve. The x-axis is the training iterations and the y-axis is the training loss. The loss should be going down.
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

x_axis_pts = np.array([idx for idx, _ in enumerate(all_losses)])

if b_calc_grad_norm:
  fig, (ax_1, ax_2) = plt.subplots(1, 2)
  
  ax_1.plot(x_axis_pts, all_losses)
  ax_1.set_xlabel("iteration")
  ax_1.set_ylabel("loss")
  ax_1.set_title("learning curve");

  ax_2.plot(x_axis_pts, all_grad_norms)
  ax_2.set_xlabel("iteration")
  ax_2.set_ylabel("norm of the gradient")
  ax_2.set_title("gradient descent");

else:
  fig, ax = plt.subplots()
  ax.plot(x_axis_pts, all_losses)
  ax.set_xlabel("iteration")
  ax.set_ylabel("loss")
  ax.set_title("learning curve");


###### **Inference**

We use the trained model to run inference on the test dataset and compute the [precision, recall](https://en.wikipedia.org/wiki/Precision_and_recall) and [f1 score](https://en.wikipedia.org/wiki/F1_score)

In [0]:
# Make prediction for one sentence.
def predict(model, sent_tensor):
  hidden = model.initHidden().to(device)

  predicted_tag_id = []

  for idx in range(sent_tensor.shape[0]):
    outputs, _ = model(sent_tensor[idx].reshape(1, sent_tensor.shape[1]), hidden)
    predicted_tag_id.append(np.argmax(outputs.detach().cpu().numpy()))

  return predicted_tag_id
 
 
model.eval()
predicted_tags = []
 
for sent_tensor in test_data_oh_list:
    sent_tensor = sent_tensor.to(device)
    predicted_tag_id = predict(model, sent_tensor)
    predicted_tags.append([id2tag[idx] for idx in predicted_tag_id])
  
   
# precision, recall, and f1 score

# Example: true_tag_list/predicted_tag_list:
#   [[‘O’, ‘O’, ‘I’, ‘N’, ...]
#    [‘I’, ‘I’, ‘O’, ‘N’, ...]],
precision, recall, f1_score = util.evaluate_result(test_tags, predicted_tags)
print("precision: {:0.4f}, recall: {:0.4f}, f1 score: {:0.4f}".format(precision, recall, f1_score))